# Extract Respiration Signal with Big-Small Model

This notebook demonstrates how to extract the respiration signal from a video using the Big-Small model.

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

subject = 'Proband05'
scenario = '101_natural_lighting'

In [ ]:
frames, meta = dataset.get_video_rgb(
    subject,
    scenario,
    num_frames=30 * 12,
    show_progress=True,
)

In [ ]:
import respiration.utils as utils
from respiration.extractor.big_small import *

device = utils.get_torch_device()
model, key_matching = load_model(device=device)
key_matching

In [ ]:
device

In [ ]:
# Preprocess the frames to big and small frames
big, small = preprocess_frames(frames)

# Garbage collection
del frames

In [ ]:
# Show the first big and small frame
import matplotlib.pyplot as plt

_, axs = plt.subplots(1, 2, figsize=(15, 5))

axs[0].imshow(big[1])
axs[0].set_title('Big Frame')

axs[1].imshow(small[1])
axs[1].set_title('Small Frame')

In [ ]:
# Create tensors from the frames
big, small = convert_to_input(big, small, device)

In [ ]:
model.eval()

with torch.no_grad():
    au_out, bvp_out, resp_out = model((big, small))

prediction = resp_out.cpu().numpy().squeeze()

In [ ]:
import respiration.analysis as analysis

respiration_gt = dataset.get_breathing_signal(subject, scenario)

# Cut the ground truth signal to the same length as the prediction
respiration_gt = respiration_gt[:len(prediction)]

compare = analysis.SignalComparator(
    prediction,
    respiration_gt,
    meta.fps,
)

In [ ]:
compare.errors()

In [ ]:
compare.signal_distances()

In [ ]:
# Plot the signals
_ = plt.figure(figsize=(20, 6))
plt.plot(compare.prediction, label='Prediction')
plt.plot(compare.ground_truth, label='Ground Truth')
plt.legend()
plt.show()